In [1]:
import os
import numpy as np
import pandas as pd
import s3fs
import zipfile
import matplotlib.pyplot as plt
from nltk import word_tokenize

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
import tiktoken
import nltk

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier

from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

from itertools import chain
from collections import Counter
import json

# Export Dataset

In [2]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

# Lister les challenges
#fs.ls("gvimont/diffusion/hackathon-minarm-2024")

# Lister les fichiers d'un challenge
fs.ls("civel/diffusion/hackathon-minarm-2024/AIVSAI")
PATH_IN = 'civel/diffusion/hackathon-minarm-2024/AIVSAI/hack_train.csv'
fs.download(PATH_IN, 'data/hack_train.csv')

[None]

In [3]:
def load_csv():
    file_path = "data/hack_train.csv"
    return pd.read_csv(filepath_or_buffer=file_path)
df = load_csv()
df

,text,label,src
0,Little disclaimer: this deals with US laws and...,1,cmv_human
1,"Read: Mentally Retarded Downs. See, we've got ...",1,cmv_human
2,"If any of you frequent rbadhistory, there is a...",1,cmv_human
3,"I believe in a flat tax system, where everyone...",1,cmv_human
4,"Edit: Ok guy's, my views have been changed on ...",1,cmv_human
...,...,...,...
56814,We consider the recovery of a source term f (x...,1,sci_gen_human
56815,"Self-supervised learning (SlfSL), aiming at le...",1,sci_gen_human
56816,Recurrent neural networks (RNNs) have achieved...,1,sci_gen_human
56817,Deep reinforcement learning (DRL) is a booming...,1,sci_gen_human


# Clean Dataframe

In [4]:
def clean_dataframe_new(df):
    df = df.drop_duplicates()
    df = df.dropna()
    
    return df
df = clean_dataframe_new(df)
df

,text,label,src
0,Little disclaimer: this deals with US laws and...,1,cmv_human
1,"Read: Mentally Retarded Downs. See, we've got ...",1,cmv_human
2,"If any of you frequent rbadhistory, there is a...",1,cmv_human
3,"I believe in a flat tax system, where everyone...",1,cmv_human
4,"Edit: Ok guy's, my views have been changed on ...",1,cmv_human
...,...,...,...
56814,We consider the recovery of a source term f (x...,1,sci_gen_human
56815,"Self-supervised learning (SlfSL), aiming at le...",1,sci_gen_human
56816,Recurrent neural networks (RNNs) have achieved...,1,sci_gen_human
56817,Deep reinforcement learning (DRL) is a booming...,1,sci_gen_human


# Save Dataset in Onyxia

In [5]:
PATH_OUT = 'linafarchado/diffusion/projet-mongroupe-hackathon/hack_train.csv'
with fs.open(PATH_OUT, 'w') as file_out:
    df.to_csv(file_out, index=False)

# Clean Dataframe

In [6]:
def clean_dataframe(df):
    df = df.drop_duplicates()
    df = df.dropna()
    
    return df

# Read From Onyxia

In [7]:
with fs.open(PATH_OUT, mode="r") as file_in:
    df = pd.read_csv(file_in)
    df = clean_dataframe(df)

In [8]:
df.head()

,text,label,src
0,Little disclaimer: this deals with US laws and...,1,cmv_human
1,"Read: Mentally Retarded Downs. See, we've got ...",1,cmv_human
2,"If any of you frequent rbadhistory, there is a...",1,cmv_human
3,"I believe in a flat tax system, where everyone...",1,cmv_human
4,"Edit: Ok guy's, my views have been changed on ...",1,cmv_human


# CNN Model

In [9]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [10]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate

def CNNModel(df):
    # Separate text and labels
    texts = df['text'].values
    labels = df['label'].values

    # Tokenization & padding
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    MAX_LEN = 100

    X = pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=MAX_LEN)

    # Invert label mapping
    label_mapping = {0: 1, 1: 0}

    if all(label in [0, 1] for label in labels):
        label_mapping = {0: 1, 1: 0}

    labels = np.array([label_mapping[label] for label in labels])

    # Split data into training, testing, and validation sets
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    # Model
    input_text = Input(shape=(MAX_LEN,), name='input_text')
    embed = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_text)
    conv = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(embed)
    pool = GlobalMaxPooling1D()(conv)
    dense1 = Dense(64, activation='relu')(pool)
    output = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=input_text, outputs=output)

    # Compile
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Training
    model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_val, y_val))

    # Evaluation
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

2024-04-24 10:00:31.575541: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 10:00:32.550676: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
CNNModel(df)

2024-04-24 10:00:46.796328: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 696 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:86:00.0, compute capability: 7.5


Epoch 1/15


I0000 00:00:1713952848.576930  199096 service.cc:145] XLA service 0x7f3d6c009600 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713952848.576978  199096 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-04-24 10:00:48.626234: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-24 10:00:48.857674: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900
2024-04-24 10:00:49.359540: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-24 10:00:49.519641: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_

  28/1066 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.5142 - loss: 0.6929

I0000 00:00:1713952850.191661  199096 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1060/1066 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6509 - loss: 0.6021

2024-04-24 10:00:57.078547: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.21GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


1066/1066 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6514 - loss: 0.6016

2024-04-24 10:00:58.864866: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.09GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


1066/1066 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.6515 - loss: 0.6015 - val_accuracy: 0.8002 - val_loss: 0.4517
Epoch 2/15
1066/1066 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.9472 - loss: 0.1567 - val_accuracy: 0.8348 - val_loss: 0.4072
Epoch 3/15
1066/1066 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9904 - loss: 0.0424 - val_accuracy: 0.8397 - val_loss: 0.4584
Epoch 4/15
1066/1066 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9945 - loss: 0.0234 - val_accuracy: 0.8385 - val_loss: 0.4959
Epoch 5/15
1066/1066 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9959 - loss: 0.0207 - val_accuracy: 0.8220 - val_loss: 0.5994
Epoch 6/15
1066/1066 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9959 - loss: 0.0170 - val_accuracy: 0.8195 - val_loss: 0.6551
Epoch 7/15
1066/1066 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9963 - loss: 0.0124 - val_accuracy: 0.8201 - val_loss: 0.7197
Epoch 8/15
1066/1066 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9962 - loss: 0.0130 - val_accur